In [1]:
"""
Uses the historical list of Kaggle competitions from here:
https://github.com/EliotAndres/kaggle-past-solutions/blob/master/competitions.yaml 

And outputs a CSV file with same information.
"""
import yaml
import pandas as pd
import json


stream = open('competitions.yaml', 'r')
cs = json.dumps(yaml.load(stream))
competitions = pd.read_json(cs, orient='records')
del competitions['userHasEntered'] # Relative to the user getting the data, irrelevant.
competitions.to_csv('competitions.csv', index=False)


In [2]:
"""
Takes a file with Kaggle competition data and outputs a file with the private leaderboards.
It uses competitionId and the internal API used in Kaggle web site (not scraping or the official API).
"""

import pandas as pd
import requests as rq
import time
import json

url = 'https://kaggle.com/c/{}/leaderboard.json?includeBeforeUser=true&includeAfterUser=false&type=private'
competitions = pd.read_csv('competitions.csv')

t = []
for _, row in competitions.iterrows():
    print("Getting data for competition:" + str(row['competitionId']))
    r = rq.get(url.format(row['competitionId']))
    res = r.json()
    if isinstance(res, dict) and res.get("beforeUser", False):
        t.extend(map(lambda x: x.update({'competitionId':row['competitionId']}) or x, res["beforeUser"]))
    if isinstance(res, dict) and res.get("afterUser", False):
        t.extend(map(lambda x: x.update({'competitionId':row['competitionId']}) or x, res["afterUser"]))
    time.sleep(1) # just to be polite to Kaggle.

teams = pd.read_json(json.dumps(t), orient='records')

#----
# Getting a new column to identify teams as (teamName attribute is not unique for members)
# the set of users.
#-----
def getUniqueTeamMembers(teamMembers):
    return sorted(set([t['profileUrl'] for t in teamMembers]))

teams["realTeam"] = teams['teamMembers'].map(getUniqueTeamMembers)


teams.to_csv('teams.csv', index=False)


# ---- Alternative scraping code (finally it was not necessary).
# from requests_html import HTMLSession
#session = HTMLSession()
#r = session.get(url + row['competitionUrl']+ '/leaderboard') 
#r.html.render(sleep=3) # Load as a Web browser will do
#teams = r.html.xpath("//tr[contains(@class, 'competition-leaderboard__row')]")
#for t in teams:
#    print(t.xpath("//td[contains(@class, 'competition-leaderboard__td-team')]/text()"))
#session.close()
#time.sleep(5)


Getting data for competition:6004
Getting data for competition:4699
Getting data for competition:4571
Getting data for competition:4657
Getting data for competition:4494
Getting data for competition:4504
Getting data for competition:4704
Getting data for competition:4521
Getting data for competition:4594
Getting data for competition:4523
Getting data for competition:4487
Getting data for competition:4493
Getting data for competition:4488
Getting data for competition:4481
Getting data for competition:4467
Getting data for competition:4471
Getting data for competition:4453
Getting data for competition:4438
Getting data for competition:4104
Getting data for competition:4407
Getting data for competition:4366
Getting data for competition:4280
Getting data for competition:4272
Getting data for competition:4066
Getting data for competition:3978
Getting data for competition:4031
Getting data for competition:4120
Getting data for competition:3973
Getting data for competition:3984
Getting data f

In [3]:
"""
"""
import pandas as pd
import numpy as np
import networkx as nx
import ast


competitions = pd.read_csv('competitions.csv', parse_dates=['deadline'])
teams = pd.read_csv('teams.csv')

G = nx.Graph()
G.add_nodes_from(competitions['competitionId'])
for key, n in G.nodes.items():
   n["type"] = "competition"
   n["name"] = competitions[competitions.competitionId == key]["competitionTitle"].values[0]
   n["url"] = competitions[competitions.competitionId == key]["competitionUrl"].values[0]


#----
# Getting a new column to identify teams as (teamName attribute is not unique for members)
# the set of users.
#-----
def getUniqueTeamMembers(teamMembers):
    """Gets a Series of strings, evaluates them as Python objects,
    gets the profile URL and then return the sorted sequence.
    """
    return sorted([t["profileUrl"] for t in ast.literal_eval(teamMembers)])

teams["realTeam"] = teams['teamMembers'].map(getUniqueTeamMembers)
print("Number of different teams:"+ str(len(teams.realTeam.transform(tuple).unique())))


G.add_nodes_from(teams.realTeam.transform(tuple).unique(), type="team")


comp_teams = np.column_stack((teams['competitionId'].values, teams['realTeam'].transform(tuple).values))
print(comp_teams)
G.add_edges_from(comp_teams)
nx.write_graphml(G, 'kaggle.xml')




Number of different teams:77655
[[6004 ('/badrobot', '/grt123', '/liaofz')]
 [6004 ('/dhammack', '/juliandewit')]
 [6004 ('/gerbenvv', '/markjan', '/timsalimans')]
 ...
 [8076 ('/ayanmaity',)]
 [8076 ('/msafi04',)]
 [8076 ('/george96uoi',)]]


In [ ]:
import matplotlib.pyplot as plt
nx.draw_networkx(G, with_labels=True)
plt.show()


In [13]:
print('Número de nodos:')
print(len(list(G.nodes())))
print('Número de aristas:')
print(len(list(G.edges())))
print('Densidad:')
print(nx.density(G))


Número de nodos:
77804
Número de aristas:
148219
Densidad:
4.897062040504622e-05


In [14]:
degree_list = []
for v in G:
    degree_list.append(G.degree(v))

print('Max degree:')
print(max(degree_list))
print('Min degree:')
print(min(degree_list))
print('Average degree:')
print(sum(degree_list)/len(degree_list))

Max degree:
7201
Min degree:
0
Average degree:
3.810061179373811


In [16]:
import networkx as nx

In [17]:
print(nx.is_connected(G))

False


In [18]:
print(nx.number_connected_components(G))

3


In [19]:
print(nx.is_directed(G))

False


In [24]:
nx.clustering(G)

{6004: 0,
 4699: 0,
 4571: 0,
 4657: 0,
 4494: 0,
 4504: 0,
 4704: 0,
 4521: 0,
 4594: 0,
 4523: 0,
 4487: 0,
 4493: 0,
 4488: 0,
 4481: 0,
 4467: 0,
 4471: 0,
 4453: 0,
 4438: 0,
 4104: 0,
 4407: 0,
 4366: 0,
 4280: 0,
 4272: 0,
 4066: 0,
 3978: 0,
 4031: 0,
 4120: 0,
 3973: 0,
 3984: 0,
 3887: 0,
 3951: 0,
 3929: 0,
 3897: 0,
 3778: 0,
 3788: 0,
 3706: 0,
 3867: 0,
 3757: 0,
 3670: 0,
 3532: 0,
 3618: 0,
 3599: 0,
 3504: 0,
 3526: 0,
 2969: 0,
 3403: 0,
 3525: 0,
 3338: 0,
 3445: 0,
 3446: 0,
 3493: 0,
 3316: 0,
 3469: 0,
 3353: 0,
 2496: 0,
 3342: 0,
 3288: 0,
 3294: 0,
 3108: 0,
 3043: 0,
 2975: 0,
 2863: 0,
 2963: 0,
 2840: 0,
 2780: 0,
 2748: 0,
 2749: 0,
 2667: 0,
 2732: 0,
 2889: 0,
 2602: 0,
 2606: 0,
 3321: 0,
 2589: 0,
 2564: 0,
 2551: 0,
 2609: 0,
 2549: 0,
 2509: 0,
 2518: 0,
 2499: 0,
 2487: 0,
 2489: 0,
 2479: 0,
 2488: 0,
 2478: 0,
 2445: 0,
 2467: 0,
 2454: 0,
 2464: 0,
 2452: 0,
 2447: 0,
 2439: 0,
 2448: 0,
 2435: 0,
 2438: 0,
 2442: 0,
 2408: 0,
 5558: 0,
 5357: 0,


In [21]:
nx.hits(G)

({6004: 5.694081061271699e-06,
  4699: 5.1793637582674784e-05,
  4571: 5.183503374965881e-06,
  4657: 4.079792643959775e-05,
  4494: 1.0989957134051187e-06,
  4504: 1.9745007105779508e-05,
  4704: 2.516408186691169e-05,
  4521: 8.579356700586533e-06,
  4594: 5.8852104279016884e-05,
  4523: 1.828173420123399e-06,
  4487: 4.406301484265026e-05,
  4493: 8.160177849133249e-06,
  4488: 1.9209719147413124e-05,
  4481: 2.4844415700545592e-05,
  4467: 2.9073402595366584e-05,
  4471: 4.2666692373080155e-05,
  4453: 1.0288258075447277e-05,
  4438: 1.2829941931104726e-05,
  4104: 1.091231153875658e-05,
  4407: 2.6646421442369154e-05,
  4366: 2.6067625158330383e-05,
  4280: 4.7989437522907726e-05,
  4272: 3.384432008990194e-05,
  4066: 4.955744342476865e-06,
  3978: 1.5166471746420555e-05,
  4031: 1.9378864738862307e-05,
  4120: 1.9829416756968046e-05,
  3973: 8.27009086261125e-06,
  3984: 9.282879747106143e-06,
  3887: 1.761848000144824e-05,
  3951: 1.1234228342627033e-05,
  3929: 4.4100469783152